In [6]:
def sacar_df(player_id,season,dict_equipo_jugador):
    player_per_stats = get_player_season(player_id, season)
    player_per_stats['GAME_DATE'] = pd.to_datetime(player_per_stats['GAME_DATE'], format='%b %d, %Y')
    player_per_stats = player_per_stats.sort_values(by='GAME_DATE')

    # Restablecer los índices del DataFrame
    player_per_stats.reset_index(drop=True, inplace=True)

    # Imprimir las primeras 5 filas del DataFrame ordenado
    # Suponiendo que ya tienes el DataFrame df_sorted ordenado por fecha
    # Eliminar las primeras 10 filas
    player_per_stats['WL'] = player_per_stats['WL'].replace({'W': 1, 'L': 0})
    # Calcular el promedio del PER de los últimos 10 partidos para cada juego
    col=['WL', 'MIN', 'FGM', 'FGA', 'FG3M', 'FG3A',  'FTM', 'FTA','OREB', 'DREB', 'AST', 'STL', 'BLK', 'TOV', 'PF','PTS', 'PLUS_MINUS']
    promedio_ultimos_10 = player_per_stats[col].shift(1).rolling(window=10, min_periods=1).mean()
    player_per_stats.update(promedio_ultimos_10)
    player_per_stats['PLAYER_ID']=player_id
    player_per_stats=player_per_stats[1:]
    player_per_stats['TEAM_ID'] = player_per_stats['PLAYER_ID'].apply(lambda x: de_que_equipo_es(x, dict_equipo_jugador))
    player_per_stats.drop(columns=['GAME_DATE'], inplace=True)
    
    return player_per_stats
   


In [7]:

dict_equipo_jugador = dfprueba.groupby('TEAM_ID')['PLAYER_ID'].apply(list).to_dict()
season='2023-24'
player_id=201935



In [2]:
import pandas as pd
import numpy as np
import requests
from nba_api.stats import endpoints
from nba_api.stats.endpoints import playergamelog

temporadas = ['2019-20','2020-21','2021-22','2022-23','2023-24']

# Iterar sobre cada temporada y obtener las estadísticas de los líderes de la liga
dfs=[]
for temporada in temporadas:
    data = endpoints.leagueleaders.LeagueLeaders(season=temporada)
    df = data.league_leaders.get_data_frame()
    # Agregar columna de temporada
    df['Temporada'] = temporada
    dfs.append(df)

# Concatenar todos los DataFrames en uno solo

# Guardar el DataFrame combinado en un archivo CSV


# Inicializar un DataFrame vacío
l=[]
df_final=pd.DataFrame()
# Iterar sobre cada player_id en l_player_id
for index, temporada in enumerate(temporadas):
    season=temporada
    
    dict_equipo_jugador = dfs[index].groupby('TEAM_ID')['PLAYER_ID'].apply(list).to_dict()
    l.append(dict_equipo_jugador)
    
    

In [5]:
# Iterar sobre cada diccionario en la lista l
# Construir el diccionario inverso
dict_jugador_equipo = {}

# Iterar sobre cada diccionario en la lista l
for dict_equipo_jugador in l:
    for team_id, player_ids in dict_equipo_jugador.items():
        for player_id in player_ids:
            if player_id not in dict_jugador_equipo:
                dict_jugador_equipo[player_id] = [team_id]
            else:
                dict_jugador_equipo[player_id].append(team_id)

# Verificar si algún jugador está asociado a más de un equipo
for player_id, team_ids in dict_jugador_equipo.items():
    if len(team_ids) > 5:
        print(f"Error: El jugador {player_id} está asociado a más de un team_id: {team_ids}")
